# Assignment: Segmenting and Clustering Neighborhoods in Toronto

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

In [1]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
import folium # map rendering library
import requests
from bs4 import BeautifulSoup

## Data Wrangling
I used Beautiful Soup based on this cool tutorial by Scott Rome: https://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/  
Read postcode data for Toronto from Wiki page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

### First define the parse table method by Scott Rome

In [2]:
def parse_html_table(table):
        n_columns = 0
        n_rows=0
        column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):

            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)

            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())

        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")

        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                          index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1

        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass

        return df

### Extract the table
Read the table and parse it using the method above

In [11]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
table = soup.find("table")

In [17]:
toronto_pc_df = parse_html_table(table)
toronto_pc_df.shape

(180, 3)

In [19]:
toronto_pc_df.head()

,Postal code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n


### Clean table up
Get rid of all the '\n' and drop the Not assigned boroughs.

In [10]:
clean_rows = pd.Series()
# Loop through the entire table and put them into a single series. Then drop all those rows that are Not assigned
for i in range(postcode_table.shape[1]):
    clean_rows = clean_rows.append(postcode_table.iloc[i].str.strip(')\n'), ignore_index = True)
    
clean_rows.drop(clean_rows[clean_rows.str.contains('Not assigned')].index, inplace=True)
clean_rows[0:5]

C:\Users\ricardog\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


0    M1A
2       
3    M2A
5       
6    M3A
dtype: object

Now let's take the first 3 characters to make the postcode column

In [7]:
postcodes = clean_rows.str[0:3]

Let's copy the rest in another series and split using '(', there are some codes that have separate neighbourhoods? (I think). So, I need to clean those up by:
1) Create a dataframe to contain the boroughs and neighborhoods, where the neighborhoods may be split into two.  
2) Remove the part that is after the ')' which contains some info I'm not sure about.  
3) Combine the neighbourhoods when there is one available.  
4) Change the '/' to ','.  

In [8]:
bor_nei = clean_rows.str[3:].str.split('(')
bor_nei_df = pd.DataFrame(item for item in bor_nei)

In [9]:
bor_nei_df[1].str.split(')')

KeyError: 1

In [ ]:
bor_nei_df

Now we can create our dataframe!

In [ ]:
toronto_df = pd.DataFrame()
toronto_df['PostCodes'] = postcodes
toronto_df['Borough','Neighborhood'] = pd.DataFrame(item for item in bor_nei)
toronto_df.head()